In [2]:
# hotel_id = 57b66e62916bb9001839f1d5 , of casino hotel
# Polynomial Regression model

import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
import calendar
import pymongo
import urllib.parse
from datetime import date
from pandas import DatetimeIndex
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
import matplotlib.pyplot as plt

In [3]:
def get_booking_data(hotel_id):
    booking_data = df_bookings[df_bookings.hotel_id == hotel_id]
    booking_data = booking_data[booking_data.cancelled == False]
    booking_data = booking_data.groupby(['stay_dt'],as_index=False)[['rooms']].sum()
    booking_data['stay_dt'] = pd.to_datetime(booking_data['stay_dt'])
    return booking_data


In [64]:
df_bookings = pd.read_csv("C:\\Users\\Rakesh\\OneDrive\\Documents\\Data\\booking.csv")
#df_hotels = pd.read_csv("C:\\Users\\Rakesh\\OneDrive\\Documents\\Data\\hotel.csv")

# Testing for casino hotel only

hotel_id = '57b66e62916bb9001839f1d5'
booking_data = get_booking_data(hotel_id)
#hotel_data = get_hotel_data(hotel_id)

C:\Users\Rakesh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4,5,10,12,15,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [65]:
def prev_yr_month(df):
    for i in df.index:
        yy = df.get_value(i,'year')
        mm = df.get_value(i,'month')
        prev_yr = df.loc[(df['year']==yy-1)&(df['month']==mm)]['rooms']
        if(prev_yr.size > 0):
            df.set_value(i,'prev_yr_bookings',prev_yr)
    return df

In [90]:
def monthwise(df):
    for i in df.index:
        stay_dt = df.get_value(i,'stay_dt')
        mm = stay_dt.month
        yy = stay_dt.year
        df.set_value(i,'month',mm)
        df.set_value(i,'year',yy)
    df = df.groupby(['month','year'],as_index=False)[['rooms']].sum()
    df['prev_yr_bookings'] = 0.0
    df = prev_yr_month(df)
    df = df[['year','month','prev_yr_bookings','rooms']]
    df = df.sort_values(by=['year', 'month'],ascending =True)
    return df

In [179]:
df_booking = monthwise(booking_data.copy())

df_train = df_booking[df_booking.year<2018]

df_test = df_booking[(df_booking.year>=2018) & (df_booking.year<2019)]

In [180]:
df_train.head()

,year,month,prev_yr_bookings,rooms
36,2010.0,5.0,0.0,1.0
0,2011.0,1.0,0.0,28.0
10,2011.0,2.0,0.0,20.0
67,2011.0,9.0,0.0,42.0
77,2011.0,10.0,0.0,140.0


In [181]:
train_x = df_train.iloc[:,1:3].values
train_y = df_train.iloc[:,3].values
test_x = df_test.iloc[:,1:3].values
test_y = df_test.iloc[:,3].values

In [182]:
month_encoder = OneHotEncoder(categorical_features = [0])

train_x = month_encoder.fit_transform(train_x).toarray()
test_x = month_encoder.transform(test_x).toarray()

In [183]:
train_x.shape

(70, 13)

In [196]:
poly = PolynomialFeatures( degree = 1 )
x_train_poly = poly.fit_transform(train_x)
lin = LinearRegression()
lin.fit(x_train_poly , train_y)

x_test_poly = poly.transform(test_x)
y_pred = lin.predict(x_test_poly)

test_pred_df = df_test


In [197]:
test_pred_df['predict'] = y_pred

C:\Users\Rakesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [198]:
test_pred_df

,year,month,prev_yr_bookings,rooms,predict
7,2018.0,1.0,854.0,2719.0,1249.879990
17,2018.0,2.0,1270.0,2190.0,1858.770040
26,2018.0,3.0,2068.0,1770.0,3065.026686
33,2018.0,4.0,1684.0,1442.0,2636.219276
41,2018.0,5.0,1107.0,1486.0,1665.386727
49,2018.0,6.0,756.0,1501.0,1118.760106
57,2018.0,7.0,926.0,1788.0,1376.362400
65,2018.0,8.0,1403.0,1757.0,2077.555947
74,2018.0,9.0,1424.0,1838.0,2017.827058
83,2018.0,10.0,1295.0,1943.0,1884.472303


In [199]:
from sklearn import metrics
print(np.sqrt(metrics.mean_squared_error(test_pred_df['rooms'], test_pred_df['predict'])))

734.304921191608
